## Question 1: TwitterReplyPercentage

In [1]:
import tweepy
from twitter_config import config

ImportError: No module named tweepy

In [2]:
def get_reply_percentage(config, username, max_tweets=100):

    auth = tweepy.OAuthHandler(config['consumer_key'], config['consumer_secret'])
    auth.set_access_token(config['access_token'], config['access_token_secret'])
    api = tweepy.API(auth)
    
    tweets = 0
    reply_status = 0
    
    for status in tweepy.Cursor(api.user_timeline, id=username).items(max_tweets):
        tweets += 1
        if status.in_reply_to_status_id is not None:
            print status.text
            reply_status += 1
    return round(((reply_status/tweets)*100),1)

In [22]:
screen_name = 'jimmyfallon'
max_tweets = 500

percentage = get_reply_percentage(config, screen_name, max_tweets)
print ("The percentage of the tweets that were replies of the user {} is {} % ".format(screen_name, percentage))

@Stranger_Things @JoseAltuve27 @KeeganMKey Plus we've got music from the great @Elbow!
I’m sorry to my college roommate. I’m the one who ate all your Bagel Bites. It was a pizza emergency. #WorldSorries
You Are What You Eat. Which Means You Eat Awesome. #MondayMotivation #FallonTonight
Swipe Right On Yourself. #MondayMotivation #FallonTonight
@HillaryClinton @MileyCyrus Hilary was great. So was Miley!! Wow!!
Thanks again to @kj_apa, @colesprouse, @CamilaMendes, @lilireinhart, and @madelainepetsch! Can't wait for Season 2!!
@ddlovato You did?!?! You did?!???? You did!!!! https://t.co/dVjejQBbRg
@catheb You will love it!! Great monologue, great guests and the Spike thing will be memorable.
Plus a Spike Jonze live dance performance featuring Mia Wasikowska &amp; @lakeithlakeith! Don't miss! #FallonTonight
@CecilyMcLeod Thank you.
.@U2 performs their brand new single "You’re The Best Thing About Me" https://t.co/5TZSuy74I5 #U2onFallon #FallonTonight
@U2 Stay up!!! DVR!!! Ha. If and when yo

## Question 2: GutenbergProject